In [ ]:
import pandas as pd
import numpy as np
#from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import time
from time import sleep

def get_distance_from_paydays(date):
    end_of_month = date.daysinmonth
    distance_to_1st = 0 if date.day >=15 else 15 - date.day
    distance_to15th = 0 if date.day < 15 else end_of_month - date.day
    return distance_to_1st + distance_to15th

def std(x): return np.std(x)


In [ ]:
df_train = pd.read_csv('../../dataset/train/train_converted.csv')
df_test  = pd.read_csv('../../dataset/test/test_converted.csv')


In [ ]:
df_train = df_train[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_WEEK_DATE','Demanda']].groupby(['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_WEEK_DATE']).sum().reset_index()
df_test = df_test[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_WEEK_DATE','Demanda']].groupby(['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_WEEK_DATE']).sum().reset_index()
df_train.replace(['',np.inf, -np.inf, np.nan],0,inplace=True)
df_test.replace(['',np.inf, -np.inf, np.nan],0,inplace=True)

dates_train_dict = pd.Series(df_train.Z_WEEK_DATE.values,index=df_train.Z_WEEK).to_dict()
dates_test_dict  = pd.Series(df_test.Z_WEEK_DATE.values,index=df_test.Z_WEEK).to_dict()
dates_total_dict = dict(dates_train_dict)
dates_total_dict.update(dates_test_dict)

list_uniques = []
for name in ['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK']:
    train_uniques = set(df_train[name].unique())
    test_uniques  = set(df_test[name].unique())
    total_uniques = list(train_uniques|test_uniques)
    print(name+' train cantidad :',len(train_uniques))
    print(name+' test  cantidad :',len(test_uniques))
    print(name+' total cantidad :',len(total_uniques))
    
    list_uniques.append(total_uniques)

In [ ]:
# importing product
from itertools import product
  
# apply product method
total_data = pd.DataFrame(list(product(list_uniques[0],list_uniques[1],list_uniques[2],list_uniques[3])))

In [ ]:
total_data.columns = ['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK']
total_data = total_data.sort_values(by=['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK'], ascending=True)
total_data.head()

In [ ]:
total_data = total_data.merge(df_train[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Demanda']],on=['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK'],how='left')

In [ ]:
total_data['Z_WEEK_DATE'] = total_data['Z_WEEK'].map(dates_total_dict)

In [ ]:
df_train_aux = total_data[total_data['Z_WEEK']<='SEMANA_50']
df_test_aux  = total_data[total_data['Z_WEEK']>'SEMANA_50']

In [ ]:

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            if str(col_type) == numerics:
                c_min = df[col].min()
                c_max = df[col].max()

                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            if str(col_type)[:5] == 'float':
                c_min = df[col].min()
                c_max = df[col].max()

                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


In [16]:
df_train_aux = reduce_mem_usage(df_train_aux)

Memory usage of dataframe is 877.00 MB
Memory usage after optimization is: 877.00 MB
Decreased by 0.0%


/tmp/ipykernel_3577/2045469029.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.float16)


In [15]:
df_test_aux = reduce_mem_usage(df_test_aux)


Memory usage of dataframe is 237.33 MB
Memory usage after optimization is: 237.33 MB
Decreased by 0.0%


/tmp/ipykernel_3577/2045469029.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.float64)


In [17]:
%timeit df_train_aux.to_pickle('../../dataset/train/train_converted_fill.pkl')


891 ms ± 79.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit df_test_aux.to_pickle('../../dataset/test/test_converted_fill.pkl')

523 ms ± 56.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
